In [1]:
import pandas as pd
import numpy as np
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
dados = pd.read_csv('logs.csv')
dados['hora'] = pd.to_datetime(dados['hora'])
dados['semana'] = dados['hora'].dt.strftime('%Y-%U')
dados = dados[dados.hora<='2022-08-01 00:00:00']
dados = dados[dados.hora>='2022-02-01 00:00:00']
notas = pd.read_csv('notas.csv')

In [2]:
dados

,hora,matricula,cod,contexto,turma,semana
7,2022-04-10 14:20:00,116551,18,Curso: 16462.1 Mecânica - 2022/1,1,2022-15
8,2022-04-10 14:19:00,116551,18,URL: pp - Apresentações de power point da disc...,1,2022-15
9,2022-04-10 14:19:00,116551,18,Curso: 16462.1 Mecânica - 2022/1,1,2022-15
10,2022-04-10 14:18:00,116551,18,Arquivo: ex - Encontro 10 - Anotações de aula,1,2022-15
11,2022-04-10 14:18:00,116551,18,Curso: 16462.1 Mecânica - 2022/1,1,2022-15
...,...,...,...,...,...,...
18373,2022-02-20 17:40:00,122726,29,URL: pl - Plano de ensino,2,2022-08
18374,2022-02-20 17:39:00,122726,29,Curso: 22578.1 Mecânica - 2022/1,2,2022-08
18375,2022-02-19 14:09:00,117572,25,Curso: 22578.1 Mecânica - 2022/1,2,2022-07
18376,2022-02-19 14:08:00,117572,25,URL: cr - Cronograma da disciplina,2,2022-07


In [3]:
categorias_ok = pd.read_excel('categorias_ok.xlsx')
dc = pd.Series(categorias_ok.categoria.values,index = categorias_ok.contexto).to_dict()
dados['categoria'] = dados['contexto'].map(dc)

In [4]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [5]:
lista_cod = dados.cod.unique().tolist()
lista_cod.sort()
lista_cat = dados.categoria.unique().tolist()
lista_cat.sort()

g1 = "Distribuição de interações com o moodle"
g2 = ""
g3 = ""
g4 = ""
g5 = ""
g6 = ""
g7 = ""
g8 = ""

l1 = [{'rowspan':4,'colspan':5},None,None,None,None,{'rowspan':1,'colspan':1,"type": "indicator"},{'rowspan':1,'colspan':1,"type": "indicator"},{'rowspan':1,'colspan':1,"type": "indicator"}]
l2 = [None,None,None,None,None,{'rowspan':1,'colspan':1,"type": "indicator"},{'rowspan':1,'colspan':1,"type": "indicator"},{'rowspan':1,'colspan':1,"type": "indicator"}]
l3 = [None,None,None,None,None,{'rowspan':2,'colspan':3,"type": "indicator"},None,None]
l4 = [None,None,None,None,None,None,None,None,]

titulos = (g1,g2,g3,g4,g5,g6,g7,g8)
specs = [l1,l2,l3,l4]

fig = make_subplots(rows=4,cols=8,specs=specs,subplot_titles=titulos,horizontal_spacing=.03,vertical_spacing=0.09)

for cod in lista_cod:
    cc = dados[dados.cod==cod]
    cn = notas[notas.cod==cod]
    cn['media'] = round((cn['nota1']+cn['nota2']+cn['nota3']+cn['nota4']+cn['nota5']+cn['nota6'])/6,2)
    ccc = cc.groupby(by=[pd.Grouper(key='hora',freq='W'),'categoria']).agg(
        semana = ('semana','min'),
        total = ('cod','count')
    ).reset_index()
    ccc['semana'] = ccc['hora'].dt.strftime('%Y-%U')

    for cat in lista_cat:
        hover = "<b>Semana: %{customdata[2]}</b>\
            <br>Tipo de interação: <b>"+cat+"</b>\
            <br>Total de interações: %{customdata[3]}"
        ccat = ccc[ccc['categoria']==cat]
        fig1 = go.Bar(x = ccat.hora,y = ccat.total,name = cat,customdata=ccat,hovertemplate=hover)
        fig.append_trace(fig1,row=1,col=1)
    
    fig2 = go.Indicator(value = cn['nota1'][cn.first_valid_index()],title='Questionário<br>Bloco 01')
    fig.append_trace(fig2,row=1,col=6)  

    fig3 = go.Indicator(value = cn['nota2'][cn.first_valid_index()],title='Questionário<br>Bloco 02')
    fig.append_trace(fig3,row=1,col=7)  

    fig4 = go.Indicator(value = cn['nota3'][cn.first_valid_index()],title='Avaliação em<br>grupo 01')
    fig.append_trace(fig4,row=1,col=8)  

    fig5 = go.Indicator(value = cn['nota4'][cn.first_valid_index()],title='Questionário<br>Bloco 03')
    fig.append_trace(fig5,row=2,col=6)  

    fig6 = go.Indicator(value = cn['nota5'][cn.first_valid_index()],title='Questionário<br>Bloco 04')
    fig.append_trace(fig6,row=2,col=7)  

    fig7 = go.Indicator(value = cn['nota6'][cn.first_valid_index()],title='Avaliação em<br>grupo 02')
    fig.append_trace(fig7,row=2,col=8) 

    fig8 = go.Indicator(value = cn['media'][cn.first_valid_index()],mode = "gauge+number",
    gauge = {'axis':{'range':[0,10]},'bar': {'color': 'black'},
        'steps': 
        [{'range': [0, 5.8], 'color': 'red'},
        {'range': [5.8,6], 'color': 'orange'},
        {'range': [6,10], 'color': 'green'}]},title='Média semestral')
    fig.add_trace(fig8,row=3,col=6)    

Ld=len(fig.data)
for k in range(24, Ld):
    fig.update_traces(visible=False, selector = k)
def create_layout_button(k, cod):
        
        visibility= [False]*24*len(lista_cod)
        for tr in range(24*k, 24*k+24):
            visibility[tr] =True
        return dict(label = cod,
                    method = 'update',
                    args = [{'visible': visibility,}])  

buttons = [create_layout_button(k, produt) for k, produt in enumerate(lista_cod)]
updatemenus = []
updatemenus.append(
    dict(active=0,
        buttons=buttons,
        showactive=True,
        x = .6,
        y = 1.1
    ))
fig.update_layout(barmode='stack') 
fig.update_layout(
    title = "Dashboard de estudante, disciplina de MecGeral, 2022-01. Selecione o código do aluno:",
    updatemenus=updatemenus,
    legend=dict(orientation="h"),
    template = "plotly_dark",
    font_family="verdana"
)
fig.write_html('dashboard.html')